In [1]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
training = pd.read_csv('training_titanic_x_y_train.csv').copy()
testing = pd.read_csv('test_titanic_x_test.csv').copy()

In [2]:
def count_nulls(dataset):
    return sum(dataset.isnull())

In [3]:
def get_name(my_list, position):
    return my_list[position][:4]

In [4]:
training['Updated_Names']=training['Name'].str.split(',').apply(get_name, position=1)
training.drop('Name', axis=1, inplace=True)

In [5]:
d=dict()
count=0
for i in set(training['Updated_Names']):
    d[i]=count/15
    count+=1
training['Names']=training['Updated_Names'].map(d)
training.drop('Updated_Names', axis=1, inplace=True)

In [6]:
training['sex']=training['Sex'].map({"female":0, "male":1})
training.drop('Sex', axis=1, inplace=True)

In [7]:
training['pclass']=training['Pclass']/3
training.drop('Pclass', axis=1, inplace=True)

In [8]:
training.loc[training['Embarked'].isnull(), 'Embarked']='S'

In [9]:
training['embarked']=training.Embarked.map({"S":0, "C":0.5, "Q":1})
training.drop('Embarked', inplace=True, axis=1)

In [10]:
training['fare']=training.Fare/training.Fare.max()
training.drop('Fare', inplace=True, axis=1)

In [11]:
training['parch']=training['Parch']/training['Parch'].max()
training.drop('Parch', inplace=True, axis=1)

In [12]:
training['sibsp']=training['SibSp']/training['SibSp'].max()
training.drop('SibSp', inplace=True, axis=1)

In [13]:
cabins=[]
for i in training.loc[training['Cabin'].notnull(), 'Cabin']:
    cabins.append(i[0])

In [14]:
set_cabins=set(cabins)

In [15]:
dictionary_cabin=dict()
count=0
for i in set_cabins:
    dictionary_cabin[i]=count/8
    count+=1
dictionary_cabin['N']=1
dictionary_cabin

{'T': 0.0,
 'E': 0.125,
 'A': 0.25,
 'F': 0.375,
 'D': 0.5,
 'G': 0.625,
 'C': 0.75,
 'B': 0.875,
 'N': 1}

In [16]:
training.loc[training.Cabin.isnull(), 'Cabin']='N'

In [17]:
def cabin_mapping(get_list, position):
    return get_list[position][0]
training['cabin']=training.Cabin.str.split(' ').apply(cabin_mapping, position=0)

In [18]:
training.drop('Cabin', axis=1, inplace=True)

In [19]:
training['cabins']=training.cabin.map(dictionary_cabin)
training.drop('cabin', inplace=True, axis=1)

In [20]:
def ticket_mapping(get_ticket, position):
    if get_ticket[position]=='LINE':
        return -1
    return int(get_ticket[position])
training['ticket']=training['Ticket'].str.split(' ').apply(ticket_mapping, position=-1)
training.drop('Ticket', inplace=True, axis=1)

In [21]:
training['tickets']=training['ticket']/training.ticket.max()
training.drop('ticket', inplace=True, axis=1)

In [22]:
training.loc[(training.Age.isnull())&(training['Survived']==0), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==0), 'Age'].median()

In [23]:
training.loc[(training.Age.isnull())&(training['Survived']==1), 'Age']=training.loc[(training.Age.notnull())&(training['Survived']==1), 'Age'].median()

In [24]:
training['age']=training.Age/80.0
training.drop('Age', inplace=True, axis=1)

In [25]:
training

,Survived,Names,sex,pclass,embarked,fare,parch,sibsp,cabins,tickets,age
0,1,0.466667,0,0.666667,0.0,0.050749,0.000000,0.125,1.0,0.073651,0.3625
1,0,0.933333,1,1.000000,0.0,0.015713,0.000000,0.000,1.0,0.000795,0.3625
2,0,0.933333,1,0.666667,0.0,0.050749,0.000000,0.000,1.0,0.080822,0.4875
3,0,0.466667,0,1.000000,0.0,0.041136,0.666667,0.000,1.0,0.112826,0.3625
4,0,0.933333,1,1.000000,0.0,0.013761,0.000000,0.000,1.0,0.126422,0.3125
...,...,...,...,...,...,...,...,...,...,...,...
663,1,0.000000,0,0.666667,0.0,0.020495,0.000000,0.000,1.0,0.004800,0.2125
664,0,0.933333,1,1.000000,1.0,0.015127,0.000000,0.000,1.0,0.120150,0.3625
665,1,0.933333,1,1.000000,0.0,0.110272,0.000000,0.000,1.0,0.000516,0.4000
666,0,0.000000,0,1.000000,0.0,0.019202,0.000000,0.000,1.0,0.002435,0.2750


In [26]:
training.drop(['Names', 'tickets'], axis = 1,inplace = True)
training

,Survived,sex,pclass,embarked,fare,parch,sibsp,cabins,age
0,1,0,0.666667,0.0,0.050749,0.000000,0.125,1.0,0.3625
1,0,1,1.000000,0.0,0.015713,0.000000,0.000,1.0,0.3625
2,0,1,0.666667,0.0,0.050749,0.000000,0.000,1.0,0.4875
3,0,0,1.000000,0.0,0.041136,0.666667,0.000,1.0,0.3625
4,0,1,1.000000,0.0,0.013761,0.000000,0.000,1.0,0.3125
...,...,...,...,...,...,...,...,...,...
663,1,0,0.666667,0.0,0.020495,0.000000,0.000,1.0,0.2125
664,0,1,1.000000,1.0,0.015127,0.000000,0.000,1.0,0.3625
665,1,1,1.000000,0.0,0.110272,0.000000,0.000,1.0,0.4000
666,0,0,1.000000,0.0,0.019202,0.000000,0.000,1.0,0.2750


In [27]:
values=training.values
from sklearn.model_selection import train_test_split
X=values[:, 1:]
Y=values[:, 0]
x_train, x_test, y_train, y_test = train_test_split(X, Y)

In [28]:
clf=LogisticRegression(n_jobs=-1, solver="saga", max_iter=20000, C=10, tol=0.000001, class_weight="balanced")
clf.fit(x_train, y_train)
clf.score(x_test, y_test)

0.8143712574850299

In [29]:
clf

LogisticRegression(C=10, class_weight='balanced', max_iter=20000, n_jobs=-1,
                   solver='saga', tol=1e-06)

In [30]:
y_pred_1 = clf.predict(x_test)
print(y_pred_1)

[1. 0. 1. 0. 0. 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 1. 1. 1. 0. 0. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 0. 0. 0. 1. 0.
 1. 0. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0. 1. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1.
 0. 1. 1. 0. 1. 1. 1. 0. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 0.
 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 1. 0.]


## Writing Predictions file

In [31]:
testing['Updated_Names']=testing['Name'].str.split(',').apply(get_name, position=1)
testing.drop('Name', axis=1, inplace=True)

In [32]:
d = dict()
count=0
for i in set(testing['Updated_Names']):
    d[i] = count/15
    count += 1
testing['Names']=testing['Updated_Names'].map(d)
testing.drop('Updated_Names', axis=1, inplace=True)

In [33]:
testing['sex']=testing['Sex'].map({"female":0, "male":1})
testing.drop('Sex', axis=1, inplace=True)

In [34]:
testing['pclass']=testing['Pclass']/3
testing.drop('Pclass', axis=1, inplace=True)

In [35]:
testing.loc[testing['Embarked'].isnull(), 'Embarked']='S'

In [36]:
testing['embarked']=testing.Embarked.map({"S":0, "C":0.5, "Q":1})
testing.drop('Embarked', inplace=True, axis=1)

In [37]:
testing['fare']=testing.Fare/testing.Fare.max()
testing.drop('Fare', inplace=True, axis=1)

In [38]:
testing['parch']=testing['Parch']/testing['Parch'].max()
testing.drop('Parch', inplace=True, axis=1)

In [39]:
testing['sibsp']=testing['SibSp']/testing['SibSp'].max()
testing.drop('SibSp', inplace=True, axis=1)

In [40]:
cabins=[]
for i in testing.loc[testing['Cabin'].notnull(), 'Cabin']:
    cabins.append(i[0])
    
set_cabins=set(cabins)

dictionary_cabin = dict()
count = 0
for i in set_cabins:
    dictionary_cabin[i] = count/8
    count += 1
dictionary_cabin['N'] = 1
dictionary_cabin

{'E': 0.0, 'A': 0.125, 'D': 0.25, 'F': 0.375, 'C': 0.5, 'B': 0.625, 'N': 1}

In [41]:
testing.loc[testing.Cabin.isnull(), 'Cabin']='N'

In [42]:
def cabin_mapping(get_list, position):
    return get_list[position][0]
testing['cabin']=testing.Cabin.str.split(' ').apply(cabin_mapping, position=0)

In [43]:
testing.drop('Cabin', axis=1, inplace=True)

In [44]:
testing['cabins']=testing.cabin.map(dictionary_cabin)
testing.drop('cabin', inplace=True, axis=1)

In [45]:
def ticket_mapping(get_ticket, position):
    if get_ticket[position]=='LINE':
        return -1
    return int(get_ticket[position])
testing['ticket']=testing['Ticket'].str.split(' ').apply(ticket_mapping, position=-1)
testing.drop('Ticket', inplace=True, axis=1)

In [46]:
testing['tickets']=testing['ticket']/testing.ticket.max()
testing.drop('ticket', inplace=True, axis=1)

In [47]:
testing['age']=testing.Age/80.0
testing.drop('Age', inplace=True, axis=1)

In [48]:
testing.drop(['Names', 'tickets'], axis = 1,inplace = True)
testing

,sex,pclass,embarked,fare,parch,sibsp,cabins,age
0,1,0.666667,0.0,0.071731,0.5,0.125,1.000,0.10000
1,0,0.333333,0.0,0.050610,0.0,0.000,0.250,0.61250
2,1,1.000000,1.0,0.015103,0.0,0.000,1.000,NaN
3,0,0.666667,0.0,0.052700,0.5,0.250,1.000,0.30000
4,1,0.333333,0.0,0.051310,0.0,0.000,0.000,0.45000
...,...,...,...,...,...,...,...,...
218,1,1.000000,0.0,0.015469,0.0,0.125,1.000,0.25000
219,1,0.333333,0.0,0.051822,0.0,0.000,0.625,0.56250
220,0,0.333333,0.5,0.212559,0.0,0.125,0.500,0.21250
221,1,1.000000,0.0,0.012590,0.0,0.000,1.000,0.53750


In [49]:
testing.isnull().sum()

sex          0
pclass       0
embarked     0
fare         0
parch        0
sibsp        0
cabins       0
age         45
dtype: int64

In [50]:
testing['age'].fillna(testing['age'].mode()[0], inplace=True)

In [51]:
testing.isnull().sum()

sex         0
pclass      0
embarked    0
fare        0
parch       0
sibsp       0
cabins      0
age         0
dtype: int64

In [52]:
testing

,sex,pclass,embarked,fare,parch,sibsp,cabins,age
0,1,0.666667,0.0,0.071731,0.5,0.125,1.000,0.10000
1,0,0.333333,0.0,0.050610,0.0,0.000,0.250,0.61250
2,1,1.000000,1.0,0.015103,0.0,0.000,1.000,0.27500
3,0,0.666667,0.0,0.052700,0.5,0.250,1.000,0.30000
4,1,0.333333,0.0,0.051310,0.0,0.000,0.000,0.45000
...,...,...,...,...,...,...,...,...
218,1,1.000000,0.0,0.015469,0.0,0.125,1.000,0.25000
219,1,0.333333,0.0,0.051822,0.0,0.000,0.625,0.56250
220,0,0.333333,0.5,0.212559,0.0,0.125,0.500,0.21250
221,1,1.000000,0.0,0.012590,0.0,0.000,1.000,0.53750


In [53]:
X_testing = testing.values
print(X_testing)

[[1.         0.66666667 0.         ... 0.125      1.         0.1       ]
 [0.         0.33333333 0.         ... 0.         0.25       0.6125    ]
 [1.         1.         1.         ... 0.         1.         0.275     ]
 ...
 [0.         0.33333333 0.5        ... 0.125      0.5        0.2125    ]
 [1.         1.         0.         ... 0.         1.         0.5375    ]
 [1.         0.66666667 0.         ... 0.         0.375      0.45625   ]]


In [54]:
ans_pred = clf.predict(X_testing)
print(ans_pred)

[0. 1. 0. 1. 1. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 1.
 0. 1. 0. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 1. 1. 0. 1.
 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0.
 1. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 0. 0. 1. 0. 0. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 0. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 1. 0.
 0. 0. 0. 0. 1. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 1. 1. 1. 0. 1. 0.
 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 0. 0. 0.
 1. 0. 0. 1. 1. 1. 1. 1. 1. 1. 0. 1. 0. 0. 0. 0. 1. 1. 0. 1. 0. 0. 1. 1.
 0. 0. 0. 0. 1. 0. 0.]


In [55]:
print(len(ans_pred))
print(testing.shape)

223
(223, 8)


In [57]:
np.savetxt("titanic_result_new.csv", ans_pred, delimiter=",", fmt='%f')